## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Powell,US,40.1578,-83.0752,51.01,50,1,10.36,clear sky
1,Ayotzintepec,MX,17.6667,-96.1333,81.32,56,87,2.80,overcast clouds
2,Bredasdorp,ZA,-34.5322,20.0403,57.20,94,96,3.40,overcast clouds
3,Bluff,NZ,-46.6000,168.3333,60.01,76,84,7.00,broken clouds
4,Saint-Philippe,RE,-21.3585,55.7679,75.20,78,0,6.91,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,Ayotzintepec,MX,17.6667,-96.1333,81.32,56,87,2.80,overcast clouds
4,Saint-Philippe,RE,-21.3585,55.7679,75.20,78,0,6.91,light rain
10,Samarai,PG,-10.6167,150.6667,78.69,89,99,11.39,overcast clouds
13,Rikitea,PF,-23.1203,-134.9692,78.58,71,11,7.11,few clouds
18,Salalah,OM,17.0151,54.0924,77.00,73,20,4.61,few clouds
...,...,...,...,...,...,...,...,...,...
686,Makakilo City,US,21.3469,-158.0858,80.01,50,1,10.36,clear sky
689,Xichang,CN,27.8964,102.2634,75.63,28,2,3.27,clear sky
691,Byron Bay,AU,-28.6500,153.6167,82.40,57,1,9.22,clear sky
694,Teahupoo,PF,-17.8333,-149.2667,80.60,78,75,3.44,broken clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#preferred_cities_df = preferred_cities_df.dropna()
#preferred_cities_df

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ayotzintepec,MX,81.32,overcast clouds,17.6667,-96.1333,
4,Saint-Philippe,RE,75.20,light rain,-21.3585,55.7679,
10,Samarai,PG,78.69,overcast clouds,-10.6167,150.6667,
13,Rikitea,PF,78.58,few clouds,-23.1203,-134.9692,
18,Salalah,OM,77.00,few clouds,17.0151,54.0924,
20,Atuona,PF,81.00,few clouds,-9.8000,-139.0333,
25,Narasannapeta,IN,86.05,overcast clouds,18.4167,84.0500,
27,Buchanan,LR,78.13,overcast clouds,5.8808,-10.0467,
29,Dogondoutchi,NE,81.73,overcast clouds,13.6393,4.0287,
30,Carnarvon,AU,82.74,clear sky,-24.8667,113.6333,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ayotzintepec,MX,81.32,overcast clouds,17.6667,-96.1333,"Hotel ""Punta del Cerro"""
4,Saint-Philippe,RE,75.20,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
10,Samarai,PG,78.69,overcast clouds,-10.6167,150.6667,Nuli Sapi
13,Rikitea,PF,78.58,few clouds,-23.1203,-134.9692,Pension Maro'i
18,Salalah,OM,77.00,few clouds,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
20,Atuona,PF,81.00,few clouds,-9.8000,-139.0333,Villa Enata
25,Narasannapeta,IN,86.05,overcast clouds,18.4167,84.0500,G.lingaraju home
27,Buchanan,LR,78.13,overcast clouds,5.8808,-10.0467,Kamaneahn Hotel Inc
29,Dogondoutchi,NE,81.73,overcast clouds,13.6393,4.0287,Hôtel Alheri Maison Alheri
30,Carnarvon,AU,82.74,clear sky,-24.8667,113.6333,Hospitality Carnarvon


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="false")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F</dd><dd>{Current Description}</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))